### Pima indian diabetes dataset
- split the dataset into 3 parts - train, test, validation
- define K (maximum number of features to be considered for the split)= square root of number of columns (K = M**1/2)
- train the randomforest classifier(RF) with the data and max_features  = K
- take a single estimator from the classifier(RF1)

###### ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

- create a new object of random forest classifier(RFa)
- use the single estimator(RF1) from the previous object in the new object(RFa)
- predict using the same dataset and compare the outputs

#### conclusion : the score of the 2 models are different (RF and RFa)

- the process is repeated after creating a new model estimator (RFb) combining multiple estimators from the first model created (RF)

In [15]:
#Import Statements
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import zero_one_loss
from statistics import mean
import copy 

In [16]:
#Data Set Reading
# list for column headers
names = ['Preg', 'Plas', 'Pres', 'Skin', 'Test', 'Mass', 'Pedi', 'Age', 'Class']
df = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv",names=names)
global N,M
N,M=df.shape
df.head()
#To convert into Target and Attributes
X_Data = df.drop("Class", axis=1)
y_Target = df["Class"]
print(N,M)

768 9


In [17]:
def data_split():    
    #To split the dataset into 3 parts   
    X_train, X_test, y_train, y_test= train_test_split(X_Data, y_Target, test_size=0.4)
    X_train, X_val, y_train, y_val= train_test_split(X_train, y_train, test_size=0.5)
    
    return X_train,X_val,X_test,y_train,y_val,y_test

 ### Algorithm 
 
1) N- No of samples in dataset 

2) M-No of features in dataset 

3) K-no of features for spliting the node=SQRT(M) 

4) Initialize 3*L array for storing error rates - Here L=300(No of decision trees in random forest) 

5) Create a for loop with 10 iterations 

      5.1) Split dataset into 3 with .3 as training,.3 validation and .4 as testing 
      5.2)Create a random forest object and fit it with training data (No of decision trees=300,max_features=SQRT(M) 
      5.3)Create an empty RF object for SFS 
      5.4)Create a copy of already created RF object for SBS 
      5.5)Create an empty RF object for SRS 
      5.6) Create a for loop with L iterations 
      
            5.6.1)Call the method for SFS ..Compute and store the error rates(In the 3*L array) of the newly created SFS random forest using testing dataset.  
            5.6.2)Call the method for SFS ..Compute and store the error rates(In the 3*L array)  of the newly created SFS random forest using testing dataset.  
            5.6.3)Call the method for SFS ..Compute and store the error rates(In the 3*L array)  of the newly created SFS random forest using testing dataset.  End both the for loop 
 
 6) Average the error rates for SFS,SBS and SRS

### As per algorithm

In [18]:
# number of decision trees
global dTree = 300
# number of features for split
K = int(M**(1/2))
# empty array for error rates
error_rates_SFS, error_rates_SBS, error_rates_SRS = [], [], []
# list to store the trees
error_rate_SFS_10,error_rate_SBS_10,error_rate_SRS_10 = [], [], []

# for loop 
for i in range(10):
    X_train, X_val, X_test, y_train, y_val, y_test = data_split()
    # Random forest model 
    classifier = RandomForestClassifier(n_estimators=dTree, max_features=K)
    # fit the classifier with data
    classifier.fit(X_train, y_train)
    # empty RF object for SFS
    classifier_SFS = RandomForestClassifier()
    error_rates_SFSMain=SFS_method(classifier_SFS,classifier)
    error_rate_SFS_10.append(error_rates_SFSMain)
    # copy of already created RF object for SBS
    classifier_SBS = copy.deepcopy(classifier)
    error_rates_SBSMain=SBS_method(classifier_SBS,classifier)
    error_rate_SBS_10.append(error_rates_SBSMain)
    # empty RF object for SRS
    classifier_SRS = RandomForestClassifier()
    error_rates_SRSMain=SRS_method(classifier_SRS,classifier)
    error_rate_SBS_10.append(error_rates_SBSMain)
   

### Code for SFS

In [19]:
def SFS_method(classifier_SFS,classifier):
    SFS_estimators = []
    error_rates_SFS = []
    for i in range(dTree):
    # creating a new list for adding new trees
        SFS_estimators.append(classifier.estimators_[i])
        # assigning the added trees to the empty RF object created for SFS
        classifier_SFS.estimators_ = SFS_estimators
        # assigning the number of estimators to the SFS object
        classifier_SFS.n_estimators = len(classifier_SFS)
        classifier_SFS.classes_ = classifier.classes_
        classifier_SFS.n_classes_ = classifier.n_classes_
        classifier_SFS.n_outputs_ = classifier.n_outputs_
        # predicting using the SFS estimator
        y_pred = classifier_SFS.predict(X_test)
        # calculating the accuracy score
        acc_score = zero_one_loss(y_test, y_pred)
        # appending score to a list
        error_rates_SFS.append(acc_score)
        
    return error_rates_SFS

In [20]:
df_SFS = pd.DataFrame(error_rate_SFS_10)

In [21]:
df_SFS

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.311688,0.308442,0.298701,0.298701,0.295455,0.301948,0.279221,0.279221,0.266234,0.269481,...,0.259740,0.262987,0.259740,0.259740,0.259740,0.259740,0.259740,0.259740,0.259740,0.259740
1,0.301948,0.327922,0.269481,0.253247,0.246753,0.237013,0.243506,0.230519,0.237013,0.243506,...,0.237013,0.237013,0.237013,0.237013,0.237013,0.237013,0.237013,0.237013,0.237013,0.237013
2,0.288961,0.295455,0.292208,0.285714,0.256494,0.253247,0.230519,0.230519,0.243506,0.246753,...,0.230519,0.227273,0.230519,0.224026,0.227273,0.227273,0.227273,0.227273,0.227273,0.227273
3,0.334416,0.295455,0.305195,0.282468,0.266234,0.272727,0.272727,0.275974,0.256494,0.266234,...,0.237013,0.237013,0.240260,0.240260,0.240260,0.240260,0.240260,0.240260,0.240260,0.240260
4,0.331169,0.301948,0.295455,0.269481,0.282468,0.269481,0.279221,0.282468,0.266234,0.275974,...,0.279221,0.279221,0.279221,0.279221,0.279221,0.279221,0.275974,0.275974,0.275974,0.275974
5,0.366883,0.321429,0.314935,0.269481,0.275974,0.256494,0.275974,0.253247,0.269481,0.240260,...,0.246753,0.250000,0.250000,0.250000,0.246753,0.246753,0.246753,0.246753,0.246753,0.246753
6,0.340909,0.301948,0.282468,0.256494,0.292208,0.250000,0.253247,0.246753,0.250000,0.246753,...,0.266234,0.266234,0.266234,0.266234,0.266234,0.266234,0.266234,0.266234,0.266234,0.272727
7,0.285714,0.279221,0.282468,0.266234,0.295455,0.262987,0.256494,0.259740,0.266234,0.259740,...,0.253247,0.256494,0.259740,0.259740,0.256494,0.256494,0.259740,0.256494,0.256494,0.256494
8,0.266234,0.285714,0.253247,0.288961,0.275974,0.272727,0.256494,0.253247,0.262987,0.256494,...,0.237013,0.233766,0.237013,0.233766,0.233766,0.233766,0.233766,0.233766,0.233766,0.233766
9,0.337662,0.340909,0.321429,0.308442,0.308442,0.318182,0.311688,0.308442,0.288961,0.308442,...,0.279221,0.282468,0.285714,0.282468,0.285714,0.282468,0.282468,0.285714,0.285714,0.285714


### Code for SBS

In [22]:
def SBS_method(classifier_SFS,classifier):
#     SBS_estimators = []
#     error_rates_SBS = []
#     for i in range(dTree):
#     # creating a new list for adding new trees
#         SFS_estimators.append(classifier.estimators_[i])
#         # assigning the added trees to the empty RF object created for SFS
#         classifier_SFS.estimators_ = SFS_estimators
#         # assigning the number of estimators to the SFS object
#         classifier_SFS.n_estimators = len(classifier_SFS)
#         classifier_SFS.classes_ = classifier.classes_
#         classifier_SFS.n_classes_ = classifier.n_classes_
#         classifier_SFS.n_outputs_ = classifier.n_outputs_
#         # predicting using the SFS estimator
#         y_pred = classifier_SFS.predict(X_test)
#         # calculating the accuracy score
#         acc_score = zero_one_loss(y_test, y_pred)
#         # appending score to a list
#         error_rates_SFS.append(acc_score)
        
#     return error_rates_SFS
    return

### Code for SRS

In [23]:
def SRS_method(classifier_SFS,classifier):
#     SRS_estimators = []
#     error_rates_SRS = []
#     for i in range(dTree):
#     # creating a new list for adding new trees
#         SFS_estimators.append(classifier.estimators_[i])
#         # assigning the added trees to the empty RF object created for SFS
#         classifier_SFS.estimators_ = SFS_estimators
#         # assigning the number of estimators to the SFS object
#         classifier_SFS.n_estimators = len(classifier_SFS)
#         classifier_SFS.classes_ = classifier.classes_
#         classifier_SFS.n_classes_ = classifier.n_classes_
#         classifier_SFS.n_outputs_ = classifier.n_outputs_
#         # predicting using the SFS estimator
#         y_pred = classifier_SFS.predict(X_test)
#         # calculating the accuracy score
#         acc_score = zero_one_loss(y_test, y_pred)
#         # appending score to a list
#         error_rates_SFS.append(acc_score)
        
#     return error_rates_SFS
    return